In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
#import plotly.graph_objects as go

#folder_name = './Looker Ecommerce BigQuery Dataset/'

In [82]:
orders = pd.read_csv('/Users/t2023-m0017/Desktop/실습/프로젝트/최종프로젝트/3차/orders.csv' ,parse_dates=['created_at','returned_at','shipped_at','delivered_at'])
order_items = pd.read_csv( '/Users/t2023-m0017/Desktop/실습/프로젝트/최종프로젝트/3차/order_items.csv')
products = pd.read_csv('/Users/t2023-m0017/Desktop/실습/프로젝트/최종프로젝트/3차/products.csv')
users = pd.read_csv('/Users/t2023-m0017/Desktop/실습/프로젝트/최종프로젝트/3차/users.csv')

In [83]:
dist_center = pd.read_csv('/Users/t2023-m0017/Desktop/실습/프로젝트/최종프로젝트/3차/distribution_centers.csv')
inv_items = pd.read_csv('/Users/t2023-m0017/Desktop/실습/프로젝트/최종프로젝트/3차/inventory_items.csv')
events = pd.read_csv( '/Users/t2023-m0017/Desktop/실습/프로젝트/최종프로젝트/3차/events.csv')

## 데이터 파악

In [84]:
# orders = orders.assign(created_at=pd.to_datetime(orders['created_at'], format='ISO8601'))

## Join: order_items, orders, products, users

In [107]:
orders[orders['order_id']==104663]

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
7718,104663,83582,Cancelled,F,2023-05-07 07:33:00+00:00,NaT,NaT,NaT,1


In [108]:
users[users['id']==83582]

,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
41039,83582,Kathryn,Ortega,kathrynortega@example.net,29,F,Hebei,3863 Johnson Brooks Suite 474,74199,Jinhua,China,39.388173,115.731465,Search,2021-11-27 07:33:00+00:00


In [109]:
order_items[order_items['order_id']==104663]

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,NaN,NaN,NaN,0.02


In [110]:
products[products['id']==14235]

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
1401,14235,0.0083,Accessories,Indestructable Aluminum Aluma Wallet - RED,marshal,0.02,Women,8425BC94A44E3D1BB3C8C026B2702C00,1


- orders.order_id = order_items.order_id
- orders.user_id = users.id
- order_items.product_id = products.id

=> order_items 기준으로 merge

1.distribution_centers.csv
칼럼:
id: 각 유통 센터의 고유 식별자.
name: 유통 센터의 이름.
latitude: 유통 센터의 위도 좌표.
longitude: 유통 센터의 경도 좌표.

2.events.csv
칼럼:
id: 각 이벤트에 대한 고유 식별자.
user_id: 이벤트와 관련된 사용자의 식별자.
sequence_number: 이벤트의 시퀀스 번호.
session_id: 이벤트가 발생한 세션의 식별자.
created_at: 이벤트가 언제 발생했는지를 나타내는 타임스탬프.
ip_address: 이벤트가 시작된 IP 주소.
city: 사건이 발생한 도시.
state: 사건이 발생한 곳.
postal_code: 행사 장소의 우편 번호.
browser: 행사 중에 사용된 웹 브라우저.
traffic_source: 행사로 이어지는 교통의 원천.
uri: 이벤트와 관련된 균일한 자원 식별자.
event_type: 기록된 이벤트 유형.

3.inventory_items.csv
칼럼:
id: 각 재고 품목에 대한 고유 식별자.
product_id: 관련 제품의 식별자.
created_at: 재고 항목이 생성되었을 때를 나타내는 타임스탬프.
sold_at: 상품이 언제 판매되었는지를 나타내는 타임스탬프.
cost: 재고 품목의 비용.
product_category: 관련 제품의 카테고리.
product_name: 관련 제품의 이름.
product_brand: 관련 제품의 브랜드.
product_retail_price: 관련 제품의 소매 가격.
product_department: 제품이 속한 부서.
product_sku: 제품의 재고 보관 단위(SKU).
product_distribution_center_id: 제품과 관련된 유통 센터의 식별자.

4.order_items.csv
칼럼:
id: 각 주문 항목에 대한 고유 식별자.
order_id: 관련 주문에 대한 식별자.
user_id: 주문한 사용자의 식별자.
product_id: 관련 제품의 식별자.
inventory_item_id: 관련 재고 품목의 식별자.
status: 주문 품목의 상태.
created_at: 주문 항목이 생성되었을 때를 나타내는 타임스탬프.
shipped_at: 주문 품목이 언제 배송되었는지를 나타내는 타임스탬프.
delivered_at: 주문 품목이 언제 배달되었는지를 나타내는 타임스탬프.
returned_at: 주문 품목이 언제 반환되었는지를 나타내는 타임스탬프.

5.orders.csv
칼럼:
order_id: 각 주문에 대한 고유 식별자.
user_id: 주문한 사용자의 식별자.
status: 주문 상태.
gender: 사용자의 성별 정보.
created_at: 주문이 언제 생성되었는지를 나타내는 타임스탬프.
returned_at: 주문이 언제 반환되었는지를 나타내는 타임스탬프.
shipped_at: 주문이 언제 배송되었는지를 나타내는 타임스탬프.
delivered_at: 주문이 언제 배달되었는지를 나타내는 타임스탬프.
num_of_item: 순서대로 있는 항목의 수.

6.products.csv
칼럼:
id: 각 제품에 대한 고유 식별자.
cost: 제품 비용.
category: 제품이 속한 범주.
name: 제품의 이름.
brand: 제품의 브랜드.
retail_price: 제품의 소매 가격.
department: 제품이 속한 부서.
sku: 제품의 재고 보관 단위(SKU).
distribution_center_id: 제품과 관련된 유통 센터의 식별자.

7.users.csv
칼럼:
id: 각 사용자에 대한 고유 식별자.
first_name: 사용자의 이름.
last_name: 사용자의 성.
email: 사용자의 이메일 주소.
age: 사용자의 나이.
gender: 사용자의 성별.
state: 사용자가 어디에 있는지 상태.
street_address: 사용자의 거리 주소.
postal_code: 사용자의 우편 번호.
city: 사용자가 위치한 도시.
country: 사용자가 위치한 국가.
latitude: 사용자의 위도 좌표.
longitude: 사용자의 경도 좌표.
traffic_source: 사용자에게 이어지는 트래픽의 출처.
created_at: 사용자 계정이 생성되었을 때를 나타내는 타임스탬프._ --> -->


In [111]:
df = orders.merge(order_items, on=['order_id', 'user_id'], suffixes=('','_order_items'))
df = df.rename(columns={'id':'order_items_id'})

df = df.merge(users, left_on=['user_id', 'gender'], right_on=['id', 'gender'], suffixes=('','_(sign up)'))
df = df.rename(columns={'created_at_(sign up)':'created_account_at'})
df = df.drop(columns='id')

df = df.merge(products, left_on='product_id', right_on='id')
df = df.drop(columns='id')

In [112]:
# 형변환
for col in df.columns:
    # if '_at' in col:    # 시간 데이터
    #     df = df.assign(**{col:pd.to_datetime(df[col], format='ISO8601')})
    #     print('[+] {0:<25} >>> datetime'.format(col))
    if '_id' in col:  # id 컬럼 -> obj형으로
        df = df.assign(**{col:df[col].astype('str')})
        print('[+] {0:<25} >>> str'.format(col))

[+] order_id                  >>> str
[+] user_id                   >>> str
[+] order_items_id            >>> str
[+] product_id                >>> str
[+] inventory_item_id         >>> str
[+] distribution_center_id    >>> str


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181759 entries, 0 to 181758
Data columns (total 39 columns):
 #   Column                    Non-Null Count   Dtype              
---  ------                    --------------   -----              
 0   order_id                  181759 non-null  object             
 1   user_id                   181759 non-null  object             
 2   status                    181759 non-null  object             
 3   gender                    181759 non-null  object             
 4   created_at                181759 non-null  datetime64[ns, UTC]
 5   returned_at               18232 non-null   datetime64[ns, UTC]
 6   shipped_at                118281 non-null  datetime64[ns, UTC]
 7   delivered_at              63841 non-null   datetime64[ns, UTC]
 8   num_of_item               181759 non-null  int64              
 9   order_items_id            181759 non-null  object             
 10  product_id                181759 non-null  object             
 11  

In [114]:
pd.set_option('display.max_columns', None)


sales_df = df.copy()
sales_df.head()

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item,order_items_id,product_id,inventory_item_id,status_order_items,created_at_order_items,shipped_at_order_items,delivered_at_order_items,returned_at_order_items,sale_price,first_name,last_name,email,age,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_account_at,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,8,5,Cancelled,F,2022-10-20 10:03:00+00:00,NaT,NaT,NaT,3,13,6998,36,Cancelled,2022-10-22 07:55:58+00:00,NaN,NaN,NaN,15.67,Vanessa,Ross,vanessaross@example.net,52,Piauí,609 Susan Row,64120-000,União,Brasil,-4.594783,-42.789899,Organic,2020-10-02 10:03:00+00:00,7.44325,Shorts,Grane Supergirl Shorts,Grane,15.67,Women,3D4893419E57449FB290647149F738D4,2
1,15775,12612,Cancelled,F,2023-11-06 05:24:00+00:00,NaT,NaT,NaT,3,22801,6998,61532,Cancelled,2023-11-06 02:30:30+00:00,NaN,NaN,NaN,15.67,Elizabeth,Miller,elizabethmiller@example.com,58,California,562 James Street Suite 269,90744,Los Angeles,United States,33.779231,-118.261542,Search,2023-06-24 05:24:00+00:00,7.44325,Shorts,Grane Supergirl Shorts,Grane,15.67,Women,3D4893419E57449FB290647149F738D4,2
2,53203,42628,Shipped,F,2023-04-05 04:04:00+00:00,NaT,2023-04-05 04:51:00+00:00,NaT,1,77348,6998,208720,Shipped,2023-04-05 03:26:10+00:00,2023-04-05 04:51:00+00:00,NaN,NaN,15.67,Lauren,Hughes,laurenhughes@example.org,22,Jilin,063 Graham Divide,130062,Shanghai,China,43.902902,125.279107,Facebook,2021-04-22 04:04:00+00:00,7.44325,Shorts,Grane Supergirl Shorts,Grane,15.67,Women,3D4893419E57449FB290647149F738D4,2
3,109313,87342,Complete,F,2023-08-23 08:17:00+00:00,NaT,2023-08-23 09:31:00+00:00,2023-08-27 10:39:00+00:00,1,158722,6998,428502,Complete,2023-08-23 08:23:33+00:00,2023-08-23 09:31:00+00:00,2023-08-27 10:39:00+00:00,NaN,15.67,Jordan,Lam,jordanlam@example.org,69,Missouri,37132 Vincent Cove Suite 513,63301,Saint Charles,United States,38.858063,-90.484397,Search,2023-01-19 08:17:00+00:00,7.44325,Shorts,Grane Supergirl Shorts,Grane,15.67,Women,3D4893419E57449FB290647149F738D4,2
4,3580,2850,Processing,F,2022-11-20 04:37:00+00:00,NaT,NaT,NaT,1,5177,6998,14006,Processing,2022-11-20 03:11:55+00:00,NaN,NaN,NaN,15.67,Lisa,Wright,lisawright@example.org,15,New Jersey,840 Christopher Branch Apt. 393,7202,Elizabeth,United States,40.650550,-74.215398,Search,2019-10-15 04:37:00+00:00,7.44325,Shorts,Grane Supergirl Shorts,Grane,15.67,Women,3D4893419E57449FB290647149F738D4,2


In [171]:
sales_df[sales_df['order_id']==]

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item,order_items_id,product_id,inventory_item_id,status_order_items,created_at_order_items,shipped_at_order_items,delivered_at_order_items,returned_at_order_items,sale_price,first_name,last_name,email,age,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_account_at,cost,category,name,brand,retail_price,department,sku,distribution_center_id
116575,10000,7954,Complete,M,2024-01-10 12:57:00+00:00,NaT,2024-01-11 22:17:00+00:00,2024-01-16 21:59:00+00:00,2,14419,22387,38937,Complete,2024-01-11 12:45:53+00:00,2024-01-11 22:17:00+00:00,2024-01-16 21:59:00+00:00,NaN,129.949997,Nathaniel,Stewart,nathanielstewart@example.org,61,Guangdong,9255 Cooper Pike Suite 509,527299,Zhangzhou,China,22.760213,111.574652,Search,2023-12-06 12:57:00+00:00,62.505949,Pants,Carhartt 100160 Men's Flame-Resistant Lined Ut...,Carhartt,129.949997,Men,194A988102EDCD4B67ACCB0110E20FD3,3
151760,10000,7954,Complete,M,2024-01-10 12:57:00+00:00,NaT,2024-01-11 22:17:00+00:00,2024-01-16 21:59:00+00:00,2,14420,19764,38939,Complete,2024-01-10 10:44:05+00:00,2024-01-11 22:17:00+00:00,2024-01-16 21:59:00+00:00,NaN,38.000000,Nathaniel,Stewart,nathanielstewart@example.org,61,Guangdong,9255 Cooper Pike Suite 509,527299,Zhangzhou,China,22.760213,111.574652,Search,2023-12-06 12:57:00+00:00,19.608000,Sweaters,Weatherproof Adult Heathland 1/4 Zip Sweater B...,Weatherproof,38.000000,Men,9F7EEFA9ACE77815CB6B9575512A0171,5


In [116]:
len(df)

181759

In [117]:
len(order_items)

181759

### 1. 거래량(매출) 분석
- 시계열
- 카테고리
- 성별: 남,여

In [118]:
sales_df.columns

Index(['order_id', 'user_id', 'status', 'gender', 'created_at', 'returned_at',
       'shipped_at', 'delivered_at', 'num_of_item', 'order_items_id',
       'product_id', 'inventory_item_id', 'status_order_items',
       'created_at_order_items', 'shipped_at_order_items',
       'delivered_at_order_items', 'returned_at_order_items', 'sale_price',
       'first_name', 'last_name', 'email', 'age', 'state', 'street_address',
       'postal_code', 'city', 'country', 'latitude', 'longitude',
       'traffic_source', 'created_account_at', 'cost', 'category', 'name',
       'brand', 'retail_price', 'department', 'sku', 'distribution_center_id'],
      dtype='object')

In [119]:
# 전처리 -> status가 'Cancelled'와 ' 'Returned' 아님 -> 수익이 되는 주문 sales_df_rev
sales_df_rev = sales_df[sales_df['status'].isin(['Shipped', 'Complete', 'Processing'])]

In [120]:
import datetime
# https://heytech.tistory.com/443
sales_df_rev['created_day'] = sales_df_rev['created_at'].dt.normalize()

/var/folders/1y/dmb1rjqn7y7ft_t08s501gxm0000gn/T/ipykernel_12753/3397747071.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [121]:
sales_df_rev['created_year'] = sales_df_rev['created_day'].dt.year

/var/folders/1y/dmb1rjqn7y7ft_t08s501gxm0000gn/T/ipykernel_12753/1975450706.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [122]:
sales_df_rev['created_month'] = sales_df_rev['created_day'].dt.month

/var/folders/1y/dmb1rjqn7y7ft_t08s501gxm0000gn/T/ipykernel_12753/3105784548.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [123]:
# 날짜별 sale_price 합계 구하기
sales_df_rev_day = sales_df_rev.groupby(['created_year','created_day']).agg({'sale_price':'sum'}).reset_index()

In [124]:
sales_df_rev_day

,created_year,created_day,sale_price
0,2019,2019-01-06 00:00:00+00:00,127.000000
1,2019,2019-01-11 00:00:00+00:00,319.000000
2,2019,2019-01-13 00:00:00+00:00,147.590000
3,2019,2019-01-14 00:00:00+00:00,64.789999
4,2019,2019-01-15 00:00:00+00:00,72.980001
...,...,...,...
1801,2024,2024-01-13 00:00:00+00:00,50098.890042
1802,2024,2024-01-14 00:00:00+00:00,70618.270111
1803,2024,2024-01-15 00:00:00+00:00,103566.160132
1804,2024,2024-01-16 00:00:00+00:00,37652.140088


In [125]:
# 월별 sale_price 합계 구하기
sales_df_rev_month = sales_df_rev.groupby(['created_year','created_month']).agg({'sale_price':'sum'}).reset_index()

In [126]:
sales_df_rev_day= sales_df_rev_day.sort_values(by='created_day', ascending=True)
sales_df_rev_day

,created_year,created_day,sale_price
0,2019,2019-01-06 00:00:00+00:00,127.000000
1,2019,2019-01-11 00:00:00+00:00,319.000000
2,2019,2019-01-13 00:00:00+00:00,147.590000
3,2019,2019-01-14 00:00:00+00:00,64.789999
4,2019,2019-01-15 00:00:00+00:00,72.980001
...,...,...,...
1801,2024,2024-01-13 00:00:00+00:00,50098.890042
1802,2024,2024-01-14 00:00:00+00:00,70618.270111
1803,2024,2024-01-15 00:00:00+00:00,103566.160132
1804,2024,2024-01-16 00:00:00+00:00,37652.140088


In [127]:
import plotly.express as px

fig = px.line(sales_df_rev_day, x = 'created_day' ,y = 'sale_price', width = 1200, height = 500, title = '일별 매출 실적')
fig.show()

In [155]:
fig = px.line(sales_df_rev_month, x = 'created_month' ,y = 'sale_price', width = 1200, height = 500, color='created_year',title = '월별 매출')
fig.show()

In [129]:
# 이동평균분석
#sales_df_rev_day = sales_df_rev.groupby(['created_day']).agg({'sale_price':'sum'}).reset_index()

sales_df_rev_day['MA7'] = sales_df_rev_day['sale_price'].rolling(window = 7).mean()
sales_df_rev_day['MA14'] = sales_df_rev_day['sale_price'].rolling(window = 14).mean()
sales_df_rev_day['MA30']= sales_df_rev_day['sale_price'].rolling(window = 30).mean()

In [130]:
sales_df_rev_day.tail(10)

,created_year,created_day,sale_price,MA7,MA14,MA30
1796,2024,2024-01-08 00:00:00+00:00,26812.170000,22506.331452,21158.299296,19389.925011
1797,2024,2024-01-09 00:00:00+00:00,24623.730054,22859.442883,21789.360013,19591.780346
1798,2024,2024-01-10 00:00:00+00:00,31255.270019,23896.994313,22480.435014,20097.347345
1799,2024,2024-01-11 00:00:00+00:00,25158.750059,24669.044319,22929.435733,20421.393013
1800,2024,2024-01-12 00:00:00+00:00,35397.040022,26404.751457,24136.355736,21055.172012
1801,2024,2024-01-13 00:00:00+00:00,50098.890042,30530.275745,26302.228593,22125.183013
1802,2024,2024-01-14 00:00:00+00:00,70618.270111,37709.160044,29872.861460,23922.324350
1803,2024,2024-01-15 00:00:00+00:00,103566.160132,48674.015777,35590.173614,26754.642354
1804,2024,2024-01-16 00:00:00+00:00,37652.140088,50535.217210,36697.330047,27379.419023
1805,2024,2024-01-17 00:00:00+00:00,39641.080089,51733.190077,37815.092195,27996.812360


In [131]:
# 이동평균분석
# 30일 -> 이전30일+오늘의 평균
a = sales_df_rev_day['sale_price'].iloc[-30:]
sum(a)/30

27996.812360060834

In [136]:
# 매출 추이
fig = px.line(sales_df_rev_day, x = 'created_day' ,y = 'MA30', width = 1200, height = 500,color='created_year', title= '30일 이동평균선')
fig.show()


In [137]:
# 매출 추이
fig = px.line(sales_df_rev_day, x = 'created_day' ,y = 'MA14', width = 1200, height = 500,color='created_year', title= '14일 이동평균선')
fig.show()


In [147]:
# 계절에 따른 매출 변화
sales_df_rev_month_sum = sales_df_rev.groupby(['created_year','created_month']).agg({'sale_price':'sum'}).reset_index()

In [148]:
sales_df_rev_month_sum

,created_year,created_month,sale_price
0,2019,1,1507.330003
1,2019,2,3598.140006
2,2019,3,5861.750011
3,2019,4,7963.050025
4,2019,5,10209.549982
...,...,...,...
56,2023,9,361234.090345
57,2023,10,412128.690386
58,2023,11,433337.350455
59,2023,12,549327.660336


In [149]:
# 계절성은 그닥...
fig = px.line(sales_df_rev_month_sum , x = 'created_month' ,y = 'sale_price', width = 1200, height = 500, title= '월별 매출 합계',color='created_year')
fig.show()

In [156]:
sales_df_rev_category = sales_df_rev.groupby(['created_month','category']).agg({'sale_price':'sum'}).reset_index()

In [157]:
sales_df_rev_category

,created_month,category,sale_price
0,1,Accessories,37406.839916
1,1,Active,41498.130015
2,1,Blazers & Jackets,28117.730062
3,1,Clothing Sets,1648.100004
4,1,Dresses,42200.890039
...,...,...,...
307,12,Suits & Sport Coats,58034.099868
308,12,Sweaters,84956.149989
309,12,Swim,55072.070065
310,12,Tops & Tees,41328.330105


In [158]:
fig = px.line(sales_df_rev_category, x = 'created_month' ,y = 'sale_price', width = 1200, height = 500, color='category',title = '월별 매출')
fig.show()

1. 시간에 따른 매출 -> 꾸준하게 증가
2. 월별 매출 -> 계절성이 없음
3. 카테고리별 매출 -> 대다수 상품이 2월에 최저 -> 그러나 매출에는 반영
4. 성별 -> 여성이 남성보다 많이 주문 
- 매출 최저점: 2월, 최고점: 연말
- 11월 효과 없음 : 블플은 안하는 듯

In [160]:
sales_df_rev_gender	= sales_df_rev.groupby(['created_day','gender']).agg({'sale_price':'sum'}).reset_index()

In [164]:
fig = px.line(sales_df_rev_gender, x = 'created_day' ,y = 'sale_price', width = 1200, height = 500, color='gender',title = '성별에 따른 일별 매출')
fig.show()

In [163]:
sales_df_rev_gender_month	= sales_df_rev.groupby(['created_month','gender']).agg({'sale_price':'sum'}).reset_index()

In [165]:
fig = px.line(sales_df_rev_gender_month, x = 'created_month' ,y = 'sale_price', width = 1200, height = 500, color='gender',title = '성별에 따른 월별 매출_연도별 합계')
fig.show()